# **Installing Pathway & Bokeh**

In [1]:
!pip install pathway bokeh --quiet

##**Model-1 BaseModel**
 **Import Necessary Libraries**


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

**Loading Dataset and displaying it's shape**

In [3]:
df=pd.read_csv('dataset.csv')
print(df.shape)


(18368, 12)


**Displaying First five lines of the dataset**

In [4]:
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


**Knowing statistics of columns in dataset**

In [5]:

df.describe()


,ID,Capacity,Latitude,Longitude,Occupancy,QueueLength,IsSpecialDay
count,18368.000000,18368.000000,18368.000000,18368.000000,18368.000000,18368.000000,18368.000000
mean,9183.500000,1605.214286,25.706547,90.751170,731.084059,4.587925,0.150915
std,5302.529208,1131.153886,1.582749,3.536636,621.164982,2.580062,0.357975
min,0.000000,387.000000,20.000035,78.000003,2.000000,0.000000,0.000000
25%,4591.750000,577.000000,26.140048,91.727995,322.000000,2.000000,0.000000
50%,9183.500000,1261.000000,26.147482,91.729511,568.000000,4.000000,0.000000
75%,13775.250000,2803.000000,26.147541,91.736172,976.000000,6.000000,0.000000
max,18367.000000,3883.000000,26.150504,91.740994,3499.000000,15.000000,1.000000


**Defining Timestamp and sort the columns in dataframe**

In [6]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values(['Timestamp']).reset_index(drop=True)

**Defining Features for streaming**

In [7]:
# For Model 1, we need SystemCodeNumber, Timestamp, Occupancy, and Capacity
# Save the selected columns to a CSV file for streaming
df[["SystemCodeNumber", "Timestamp", "Occupancy", "Capacity"]].to_csv("parking_stream_model1.csv", index=False)

print(f"Data prepared for {df['SystemCodeNumber'].nunique()} unique parking lots")
print(f"Total records: {len(df)}")

Data prepared for 14 unique parking lots
Total records: 18368


**Define Schema for Streaming Data & Load the data as a simulated stream using Pathway's replay_csv function**

In [8]:
# Define Schema for Streaming Data
class ParkingSchemaModel1(pw.Schema):
    SystemCodeNumber: str    # Unique identifier for each parking lot
    Timestamp: str          # Timestamp of the observation
    Occupancy: int          # Number of occupied parking spots
    Capacity: int           # Total parking capacity at the location

# Load Data as Simulated Stream
# Load the data as a simulated stream using Pathway's replay_csv function
data = pw.demo.replay_csv("parking_stream_model1.csv",
                         schema=ParkingSchemaModel1,
                         input_rate=100)  # Reduced rate for better real-time simulation

# Parse Timestamp
fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    occupancy_rate = data.Occupancy / data.Capacity  # Calculate occupancy rate
)

**Define pricing parameters as constants**

In [9]:

# Model 1 - Baseline Linear Pricing Implementation
# Define pricing parameters as constants
BASE_PRICE = 10.0
ALPHA = 0.05  # Price adjustment factor (you can tune this)
MIN_PRICE_MULTIPLIER = 0.5  # Minimum price is 50% of base
MAX_PRICE_MULTIPLIER = 2.0  # Maximum price is 300% of base

# Global dictionary to track prices for each lot
lot_prices = {}

**Defining function to calculate the price using model-1 formula**

---
*Price(t+1) = Price(t) + α × (Occupancy/Capacity)*


In [10]:

def calculate_model1_price(lot_id: str, occupancy_rate: float) -> float:
    """
    Calculate price using Model 1 formula for a specific parking lot
    Model 1 Formula: Price(t+1) = Price(t) + α × (Occupancy/Capacity)

    Args:
        lot_id: SystemCodeNumber identifying the parking lot
        occupancy_rate: Current occupancy rate (Occupancy/Capacity)

    Returns:
        Updated price for the parking lot
    """
    # Get current price (or initialize with base price)
    current_price = lot_prices.get(lot_id, BASE_PRICE)

    # Apply Model 1 formula
    new_price = current_price + ALPHA * occupancy_rate

    # Apply reasonable bounds (price shouldn't go below base/2 or above base*3)
    new_price = max(BASE_PRICE * MIN_PRICE_MULTIPLIER,
                   min(new_price, BASE_PRICE * MAX_PRICE_MULTIPLIER))

    # Update stored price
    lot_prices[lot_id] = new_price

    return new_price

In [11]:

# Apply Model 1 Pricing Logic
# Process each data point and calculate dynamic pricing
model1_prices = data_with_time.with_columns(
    # Calculate the new price using Model 1 formula
    price = pw.apply(calculate_model1_price,
                    pw.this.SystemCodeNumber,
                    pw.this.occupancy_rate),

    # Add additional useful columns for analysis
    price_change_factor = pw.this.occupancy_rate * ALPHA,
    pricing_model = pw.cast(str, "Model1_Baseline")
)

# Create Summary Statistics per Parking Lot
# This gives us a view of how each lot's pricing behaves
lot_summary = (
    model1_prices
    .groupby(pw.this.SystemCodeNumber)
    .reduce(
        lot_id=pw.this.SystemCodeNumber,
        avg_price=pw.reducers.avg(pw.this.price),
        max_price=pw.reducers.max(pw.this.price),
        min_price=pw.reducers.min(pw.this.price),
        avg_occupancy_rate=pw.reducers.avg(pw.this.occupancy_rate),
        total_updates=pw.reducers.count()
    )
    .with_columns(
        price_volatility=pw.this.max_price - pw.this.min_price
    )
)

# **Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure**

In [12]:
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (model1_prices) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = model1_prices.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [13]:
# Real-time Output Setup
# Set up output to see the pricing in real-time
def print_pricing_update(row):
    """Callback function to print pricing updates"""
    print(f"Lot {row['SystemCodeNumber']}: Price=${row['price']:.2f}, "
          f"Occupancy={row['occupancy_rate']:.2f}, Time={row['Timestamp']}")

# Apply the callback to see real-time updates
model1_prices.debug("Real-time Pricing Updates")

# Save Results for Analysis
# Save the pricing results to CSV for further analysis
pw.io.csv.write(model1_prices, "model1_pricing_results.csv")
pw.io.csv.write(lot_summary, "model1_lot_summary.csv")

# Model Parameters and Configuration
print("\n=== MODEL 1 CONFIGURATION ===")
print(f"Base Price: ${BASE_PRICE}")
print(f"Alpha (adjustment factor): {ALPHA}")
print(f"Price Bounds: ${BASE_PRICE * MIN_PRICE_MULTIPLIER:.2f} - ${BASE_PRICE * MAX_PRICE_MULTIPLIER:.2f}")
print(f"Formula: Price(t+1) = Price(t) + {ALPHA} × (Occupancy/Capacity)")

# Run the Pipeline
if __name__ == "__main__":
    # Run the Pathway pipeline
    pw.run()

    print("\n=== MODEL 1 EXECUTION COMPLETE ===")
    print("Check 'model1_pricing_results.csv' for detailed results")
    print("Check 'model1_lot_summary.csv' for lot-wise summary")

Output()

/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(



=== MODEL 1 CONFIGURATION ===
Base Price: $10.0
Alpha (adjustment factor): 0.05
Price Bounds: $5.00 - $20.00
Formula: Price(t+1) = Price(t) + 0.05 × (Occupancy/Capacity)



=== MODEL 1 EXECUTION COMPLETE ===
Check 'model1_pricing_results.csv' for detailed results
Check 'model1_lot_summary.csv' for lot-wise summary


# **Model 2: Demand-Based Price Function**

**Importing libraries**

In [14]:
import pandas as pd
import pathway as pw
import numpy as np
from typing import Dict, Optional
from datetime import datetime, time

**Data Preparation for Model 2**

In [15]:
# Step 1: Data Preparation for Model 2
# Read the original dataset and prepare it for streaming
df = pd.read_csv("dataset.csv")  # Replace with your actual dataset path

# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values(['SystemCodeNumber', 'Timestamp']).reset_index(drop=True)


In [16]:

# Add synthetic columns for Model 2 (in real implementation, these would come from actual data)
np.random.seed(42)  # For reproducible results
df['QueueLength'] = np.random.choice([0, 1, 2, 3, 4, 5], len(df), p=[0.3, 0.2, 0.2, 0.15, 0.1, 0.05])
df['TrafficLevel'] = np.random.choice(['Low', 'Medium', 'High'], len(df), p=[0.3, 0.4, 0.3])
df['VehicleType'] = np.random.choice(['Bike', 'Car', 'Truck'], len(df), p=[0.2, 0.7, 0.1])

**Preparing Features for model-2**

In [17]:

# For Model 2, we need additional columns
model2_columns = ["SystemCodeNumber", "Timestamp", "Occupancy", "Capacity",
                 "QueueLength", "TrafficLevel", "VehicleType"]
df[model2_columns].to_csv("parking_stream_model2.csv", index=False)

print(f"Data prepared for Model 2 with {df['SystemCodeNumber'].nunique()} unique parking lots")
print(f"Total records: {len(df)}")

Data prepared for Model 2 with 14 unique parking lots
Total records: 18368


**Define Schema for Model 2 Streaming Data**

In [18]:
# Step 2: Define Schema for Model 2 Streaming Data
class ParkingSchemaModel2(pw.Schema):
    SystemCodeNumber: str    # Unique identifier for each parking lot
    Timestamp: str          # Timestamp of the observation
    Occupancy: int          # Number of occupied parking spots
    Capacity: int           # Total parking capacity at the location
    QueueLength: int        # Number of vehicles waiting in queue
    TrafficLevel: str       # Traffic level: Low, Medium, High
    VehicleType: str        # Vehicle type: Bike, Car, Truck

**Load Data as Simulated Stream**

In [19]:

# Step 3: Load Data as Simulated Stream
data = pw.demo.replay_csv("parking_stream_model2.csv",
                         schema=ParkingSchemaModel2,
                         input_rate=1000)  # Slower rate for complex processing

**Parse Timestamp and Add Time-based**

In [20]:
# Step 4: Parse Timestamp and Add Time-based Features
fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    occupancy_rate = data.Occupancy / data.Capacity
)

**Base pricing parameters**

In [21]:

# Step 5: Model 2 - Advanced Demand-Based Pricing Parameters
# Base pricing parameters
BASE_PRICE = 10.0
MIN_PRICE_MULTIPLIER = 0.5  # Minimum price is 50% of base
MAX_PRICE_MULTIPLIER = 2.0  # Maximum price is 200% of base

# Demand function coefficients
ALPHA = 0.4      # Occupancy weight
BETA = 0.15      # Queue length weight
GAMMA = 0.1      # Traffic weight (negative impact)
DELTA = 0.25     # Special day weight
EPSILON = 0.1    # Vehicle type weight
LAMBDA = 0.8     # Price adjustment sensitivity


**Creating Multipliers for pricing variations**

In [22]:

# Time-based multipliers
TIME_MULTIPLIERS = {
    'morning': 1.2,    # 6-10 AM: Morning hike
    'afternoon': 1.0,  # 10 AM-4 PM: Normal
    'evening': 1.15,   # 4-8 PM: Evening hike
    'night': 0.9       # 8 PM-6 AM: Low demand
}

# Day-based multipliers
DAY_MULTIPLIERS = {
    'weekday': 1.0,    # Monday-Friday: Normal
    'weekend': 1.3     # Saturday-Sunday: Hike
}

# Vehicle type weights
VEHICLE_WEIGHTS = {
    'Bike': 0.8,      # Bikes: Low
    'Car': 1.0,       # Cars: Little hike (baseline)
    'Truck': 1.4      # Trucks: More hike
}

# Traffic level weights
TRAFFIC_WEIGHTS = {
    'Low': 0.8,       # Low traffic: decrease
    'Medium': 1.0,    # Medium traffic: normal
    'High': 1.25      # High traffic: increase
}

**Queue length weights (normalized)**

In [23]:

# Queue length weights (normalized)
def get_queue_weight(queue_length: int) -> float:
    """Convert queue length to weight"""
    if queue_length == 0:
        return 0.0      # No queue: no hike
    elif queue_length <= 2:
        return 0.3      # Small queue: little hike
    elif queue_length <= 4:
        return 0.6      # Medium queue: moderate hike
    else:
        return 1.0      # Large queue: more hike


In [24]:

# Global dictionary to track prices for each lot
lot_prices_model2 = {}

**Defining Helper functions for time-based featurest**

In [25]:

# Step 6: Helper Functions for Time-based Features
def get_time_of_day(timestamp_str: str) -> str:
    """Determine time of day category"""
    try:
        dt = datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S")
        hour = dt.hour

        if 6 <= hour < 10:
            return 'morning'
        elif 10 <= hour < 16:
            return 'afternoon'
        elif 16 <= hour < 20:
            return 'evening'
        else:
            return 'night'
    except:
        return 'afternoon'  # Default

In [26]:
def get_day_type(timestamp_str: str) -> str:
    """Determine if it's weekday or weekend"""
    try:
        dt = datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S")
        if dt.weekday() < 5:  # Monday = 0, Sunday = 6
            return 'weekday'
        else:
            return 'weekend'
    except:
        return 'weekday'  # Default



In [27]:
def is_special_day(timestamp_str: str) -> bool:
    """Check if it's a special day (predefined special dates and calculated patterns)"""
    try:
        dt = datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S")

        # Define fixed special days as a list of (month, day) tuples
        special_days = [
            (1, 1),   # New Year's Day
            (1, 26),  # Republic Day (India)
            (2, 14),  # Valentine's Day
            (3, 8),   # Women's Day
            (4, 14),  # Good Friday (approximate)
            (5, 1),   # Labor Day
            (7, 4),   # Independence Day (US)
            (8, 15),  # Independence Day (India)
            (10, 2),  # Gandhi Jayanti (India)
            (10, 31), # Halloween
            (11, 14), # Children's Day (India)
            (12, 25), # Christmas Day
            (12, 31), # New Year's Eve
        ]

        # Check if current date matches any fixed special day
        current_date = (dt.month, dt.day)
        if current_date in special_days:
            return True

        # Calculate dynamic special days using datetime patterns

        # 1. First Monday of September (Labor Day in some countries)
        if dt.month == 9:
            first_monday = None
            for day in range(1, 8):
                test_date = datetime(dt.year, 9, day)
                if test_date.weekday() == 0:  # Monday
                    first_monday = day
                    break
            if first_monday and dt.day == first_monday:
                return True

        # 2. Fourth Thursday of November (Thanksgiving - US)
        if dt.month == 11:
            fourth_thursday = None
            thursday_count = 0
            for day in range(1, 32):
                try:
                    test_date = datetime(dt.year, 11, day)
                    if test_date.weekday() == 3:  # Thursday
                        thursday_count += 1
                        if thursday_count == 4:
                            fourth_thursday = day
                            break
                except ValueError:
                    break
            if fourth_thursday and dt.day == fourth_thursday:
                return True

        # 3. Second Sunday of May (Mother's Day)
        if dt.month == 5:
            second_sunday = None
            sunday_count = 0
            for day in range(1, 32):
                try:
                    test_date = datetime(dt.year, 5, day)
                    if test_date.weekday() == 6:  # Sunday
                        sunday_count += 1
                        if sunday_count == 2:
                            second_sunday = day
                            break
                except ValueError:
                    break
            if second_sunday and dt.day == second_sunday:
                return True

        # 4. Third Sunday of June (Father's Day)
        if dt.month == 6:
            third_sunday = None
            sunday_count = 0
            for day in range(1, 32):
                try:
                    test_date = datetime(dt.year, 6, day)
                    if test_date.weekday() == 6:  # Sunday
                        sunday_count += 1
                        if sunday_count == 3:
                            third_sunday = day
                            break
                except ValueError:
                    break
            if third_sunday and dt.day == third_sunday:
                return True

        # 5. Spring Equinox (around March 20-21)
        if dt.month == 3 and dt.day in [20, 21]:
            return True

        # 6. Summer Solstice (around June 21-22)
        if dt.month == 6 and dt.day in [21, 22]:
            return True

        # 7. Autumn Equinox (around September 22-23)
        if dt.month == 9 and dt.day in [22, 23]:
            return True

        # 8. Winter Solstice (around December 21-22)
        if dt.month == 12 and dt.day in [21, 22]:
            return True

        # Regular pattern-based special days
        # First day of every month
        if dt.day == 1:
            return True

        # Last day of every month
        if dt.month == 12:
            next_month = dt.replace(year=dt.year + 1, month=1, day=1)
        else:
            next_month = dt.replace(month=dt.month + 1, day=1)

        last_day_of_month = (next_month - datetime.timedelta(days=1)).day
        if dt.day == last_day_of_month:
            return True

        # Mid-month (15th) - often payday or special
        if dt.day == 15:
            return True

        # Weekend days (Saturday and Sunday)
        if dt.weekday() >= 5:  # 5=Saturday, 6=Sunday
            return True

        # Day before and after major holidays (bridge days)
        for month, day in special_days:
            try:
                special_date = datetime(dt.year, month, day)
                day_before = special_date - datetime.timedelta(days=1)
                day_after = special_date + datetime.timedelta(days=1)

                if dt.date() == day_before.date() or dt.date() == day_after.date():
                    return True
            except ValueError:
                continue

        return False

    except:
        return False


**Method for Demand Caluculation**

In [28]:
# Step 7: Advanced Demand Calculation Function
def calculate_demand_score(occupancy_rate: float, queue_length: int,
                          traffic_level: str, vehicle_type: str,
                          timestamp_str: str) -> float:
    """
    Calculate comprehensive demand score based on multiple factors

    Demand = α·(Occupancy/Capacity) + β·QueueLength - γ·Traffic + δ·IsSpecialDay + ε·VehicleTypeWeight
    """

    # Base demand from occupancy
    occupancy_component = ALPHA * occupancy_rate

    # Queue length component (normalized)
    queue_component = BETA * get_queue_weight(queue_length)

    # Traffic component (note: we subtract because high traffic reduces attractiveness)
    traffic_weight = TRAFFIC_WEIGHTS.get(traffic_level, 1.0)
    traffic_component = -GAMMA * (traffic_weight - 1.0)  # Normalize around 0

    # Special day component
    special_day_component = DELTA if is_special_day(timestamp_str) else 0.0

    # Vehicle type component
    vehicle_weight = VEHICLE_WEIGHTS.get(vehicle_type, 1.0)
    vehicle_component = EPSILON * (vehicle_weight - 1.0)  # Normalize around 0

    # Time-based adjustments
    time_multiplier = TIME_MULTIPLIERS.get(get_time_of_day(timestamp_str), 1.0)
    day_multiplier = DAY_MULTIPLIERS.get(get_day_type(timestamp_str), 1.0)

    # Calculate base demand
    base_demand = (occupancy_component + queue_component + traffic_component +
                  special_day_component + vehicle_component)

    # Apply time and day multipliers
    adjusted_demand = base_demand * time_multiplier * day_multiplier

    return adjusted_demand

**Normalize demand score to reasonable range**

In [29]:
def normalize_demand(demand_score: float) -> float:
    """Normalize demand score to reasonable range"""
    # Apply sigmoid-like function to bound the demand
    normalized = np.tanh(demand_score)  # Bounds between -1 and 1
    return normalized

# **Method for pricing calculation by using demand features**

In [30]:

def calculate_model2_price(lot_id: str, occupancy_rate: float, queue_length: int,
                          traffic_level: str, vehicle_type: str, timestamp_str: str) -> float:
    """
    Calculate price using Model 2 advanced demand-based formula

    Price(t) = BasePrice * (1 + λ * NormalizedDemand)
    """

    # Calculate demand score
    demand_score = calculate_demand_score(occupancy_rate, queue_length,
                                        traffic_level, vehicle_type, timestamp_str)

    # Normalize demand
    normalized_demand = normalize_demand(demand_score)

    # Calculate new price
    price_multiplier = 1 + LAMBDA * normalized_demand
    new_price = BASE_PRICE * price_multiplier

    # Apply bounds (0.5x to 2.0x base price)
    new_price = max(BASE_PRICE * MIN_PRICE_MULTIPLIER,
                   min(new_price, BASE_PRICE * MAX_PRICE_MULTIPLIER))

    # Store price for this lot
    lot_prices_model2[lot_id] = new_price

    return new_price

In [31]:
# Step 8: Apply Model 2 Pricing Logic
model2_prices = data_with_time.with_columns(
    # Calculate demand score components
    time_of_day = pw.apply(get_time_of_day, pw.this.Timestamp),
    day_type = pw.apply(get_day_type, pw.this.Timestamp),
    is_holiday = pw.apply(is_special_day, pw.this.Timestamp),

    # Calculate the new price using Model 2 formula
    price = pw.apply(calculate_model2_price,
                    pw.this.SystemCodeNumber,
                    pw.this.occupancy_rate,
                    pw.this.QueueLength,
                    pw.this.TrafficLevel,
                    pw.this.VehicleType,
                    pw.this.Timestamp),

    # Calculate demand score for analysis
    demand_score = pw.apply(calculate_demand_score,
                           pw.this.occupancy_rate,
                           pw.this.QueueLength,
                           pw.this.TrafficLevel,
                           pw.this.VehicleType,
                           pw.this.Timestamp),

    # Add pricing model identifier
    pricing_model = pw.cast(str, "Model2_DemandBased")
)


**Making Summary Statistics**

In [32]:
# Step 9: Create Advanced Summary Statistics
lot_summary_model2 = (
    model2_prices
    .groupby(pw.this.SystemCodeNumber)
    .reduce(
        lot_id=pw.this.SystemCodeNumber,
        avg_price=pw.reducers.avg(pw.this.price),
        max_price=pw.reducers.max(pw.this.price),
        min_price=pw.reducers.min(pw.this.price),
        avg_occupancy_rate=pw.reducers.avg(pw.this.occupancy_rate),
        avg_demand_score=pw.reducers.avg(pw.this.demand_score),
        total_updates=pw.reducers.count(),
        price_volatility=pw.reducers.max(pw.this.price) - pw.reducers.min(pw.this.price)
    )
)

# Time-based analysis
time_based_analysis = (
    model2_prices
    .groupby(pw.this.time_of_day)
    .reduce(
        time_period=pw.this.time_of_day,
        avg_price=pw.reducers.avg(pw.this.price),
        avg_demand=pw.reducers.avg(pw.this.demand_score),
        count=pw.reducers.count()
    )
)

# Day type analysis
day_type_analysis = (
    model2_prices
    .groupby(pw.this.day_type)
    .reduce(
        day_category=pw.this.day_type,
        avg_price=pw.reducers.avg(pw.this.price),
        avg_demand=pw.reducers.avg(pw.this.demand_score),
        count=pw.reducers.count()
    )
)

# Vehicle type analysis
vehicle_analysis = (
    model2_prices
    .groupby(pw.this.VehicleType)
    .reduce(
        vehicle_type=pw.this.VehicleType,
        avg_price=pw.reducers.avg(pw.this.price),
        avg_demand=pw.reducers.avg(pw.this.demand_score),
        count=pw.reducers.count()
    )
)

In [33]:
# Step 10: Real-time Output Setup
def print_model2_update(row):
    """Callback function to print Model 2 pricing updates"""
    print(f"Model2 - Lot {row['SystemCodeNumber']}: Price=${row['price']:.2f}, "
          f"Demand={row['demand_score']:.3f}, Occupancy={row['occupancy_rate']:.2f}, "
          f"Queue={row['QueueLength']}, Traffic={row['TrafficLevel']}, "
          f"Vehicle={row['VehicleType']}, Time={row['time_of_day']}")

# Apply the callback to see real-time updates
model2_prices.debug("Model 2 Real-time Updates")

Traceback (most recent call last):
 File "/usr/local/lib/python3.11/dist-packages/pyviz_comms/__init__.py", line 341, in _handle_msg
 self._on_msg(msg)
 File "/usr/local/lib/python3.11/dist-packages/panel/viewable.py", line 500, in _on_msg
 doc.unhold()
 File "/usr/local/lib/python3.11/dist-packages/bokeh/document/document.py", line 776, in unhold
 self.callbacks.unhold()
 File "/usr/local/lib/python3.11/dist-packages/bokeh/document/callbacks.py", line 441, in unhold
 self.trigger_on_change(event)
 File "/usr/local/lib/python3.11/dist-packages/bokeh/document/callbacks.py", line 418, in trigger_on_change
 invoke_with_curdoc(doc, event.callback_invoker)
 File "/usr/local/lib/python3.11/dist-packages/bokeh/document/callbacks.py", line 453, in invoke_with_curdoc
 return f()
 ^^^
 File "/usr/local/lib/python3.11/dist-packages/bokeh/util/callback_manager.py", line 174, in invoke
 callback(attr, old, new)
 File "/usr/local/lib/python3.11/dist-packages/panel/reactive.py", line 561, in _comm_change
 state._handle_exception(e)
 File "/usr/local/lib/python3.11/dist-packages/panel/io/state.py", line 488, in _handle_exception
 raise exception
 File "/usr/local/lib/python3.11/dist-packages/panel/reactive.py", line 559, in _comm_change
 self._schedule_change(doc, comm)
 File "/usr/local/lib/python3.11/dist-packages/panel/reactive.py", line 541, in _schedule_change
 self._change_event(doc)
 File "/usr/local/lib/python3.11/dist-packages/panel/reactive.py", line 537, in _change_event
 self._process_events(events)
 File "/usr/local/lib/python3.11/dist-packages/panel/widgets/tables.py", line 1381, in _process_events
 return super()._process_events(events)
 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 File "/usr/local/lib/python3.11/dist-packages/panel/reactive.py", line 1453, in _process_events
 self._process_data(events.pop('data'))
 File "/usr/local/lib/python3.11/dist-packages/panel/widgets/tables.py", line 1471, in _process_data
 return super()._process_data(data)
 ^^^^^^^^^^^^^^^^^^^^^^^^^^^
 File "/usr/local/lib/python3.11/dist-packages/panel/reactive.py", line 1426, in _process_data
 self._update_column(col, converted)
 File "/usr/local/lib/python3.11/dist-packages/panel/widgets/tables.py", line 1786, in _update_column
 self.value.loc[index, column] = array
 ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^
 File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexing.py", line 911, in __setitem__
 iloc._setitem_with_indexer(indexer, value, self.name)
 File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexing.py", line 1942, in _setitem_with_indexer
 self._setitem_with_indexer_split_path(indexer, value, name)
 File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexing.py", line 1998, in _setitem_with_indexer_split_path
 raise ValueError(
ValueError: Must have equal len keys and value when setting with an iterable
 


 Traceback (most recent call last):
 File "/usr/local/lib/python3.11/dist-packages/pyviz_comms/__init__.py", line 341, in _handle_msg
 self._on_msg(msg)
 File "/usr/local/lib/python3.11/dist-packages/panel/viewable.py", line 500, in _on_msg
 doc.unhold()
 File "/usr/local/lib/python3.11/dist-packages/bokeh/document/document.py", line 776, in unhold
 self.callbacks.unhold()
 File "/usr/local/lib/python3.11/dist-packages/bokeh/document/callbacks.py", line 441, in unhold
 self.trigger_on_change(event)
 File "/usr/local/lib/python3.11/dist-packages/bokeh/document/callbacks.py", line 418, in trigger_on_change
 invoke_with_curdoc(doc, event.callback_invoker)
 File "/usr/local/lib/python3.11/dist-packages/bokeh/document/callbacks.py", line 453, in invoke_with_curdoc
 return f()
 ^^^
 File "/usr/local/lib/python3.11/dist-packages/bokeh/util/callback_manager.py", line 174, in invoke
 callback(attr, old, new)
 File "/usr/local/lib/python3.11/dist-packages/panel/reactive.py", line 561, in _comm_change
 state._handle_exception(e)
 File "/usr/local/lib/python3.11/dist-packages/panel/io/state.py", line 488, in _handle_exception
 raise exception
 Fil

<pathway.Table schema={'SystemCodeNumber': <class 'str'>, 'Timestamp': <class 'str'>, 'Occupancy': <class 'int'>, 'Capacity': <class 'int'>, 'QueueLength': <class 'int'>, 'TrafficLevel': <class 'str'>, 'VehicleType': <class 'str'>, 't': <class 'pathway.internals.datetime_types.DateTimeNaive'>, 'occupancy_rate': <class 'float'>, 'time_of_day': <class 'str'>, 'day_type': <class 'str'>, 'is_holiday': <class 'bool'>, 'price': <class 'float'>, 'demand_score': <class 'float'>, 'pricing_model': <class 'str'>}>

In [37]:
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (model1_prices) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = model2_prices.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

# **Save Results for Analysis**

In [34]:

# Step 11: Save Results for Analysis
pw.io.csv.write(model2_prices, "model2_pricing_results.csv")
pw.io.csv.write(lot_summary_model2, "model2_lot_summary.csv")
pw.io.csv.write(time_based_analysis, "model2_time_analysis.csv")
pw.io.csv.write(day_type_analysis, "model2_day_analysis.csv")
pw.io.csv.write(vehicle_analysis, "model2_vehicle_analysis.csv")

In [35]:

# Step 12: Model 2 Configuration Display
print("\n=== MODEL 2 CONFIGURATION ===")
print(f"Base Price: ${BASE_PRICE}")
print(f"Price Range: ${BASE_PRICE * MIN_PRICE_MULTIPLIER:.2f} - ${BASE_PRICE * MAX_PRICE_MULTIPLIER:.2f}")
print(f"Demand Coefficients:")
print(f"  α (Occupancy): {ALPHA}")
print(f"  β (Queue): {BETA}")
print(f"  γ (Traffic): {GAMMA}")
print(f"  δ (Special Day): {DELTA}")
print(f"  ε (Vehicle Type): {EPSILON}")
print(f"  λ (Price Sensitivity): {LAMBDA}")
print(f"Formula: Demand = α·(Occ/Cap) + β·Queue - γ·Traffic + δ·Holiday + ε·Vehicle")
print(f"Price = BasePrice × (1 + λ × NormalizedDemand)")



=== MODEL 2 CONFIGURATION ===
Base Price: $10.0
Price Range: $5.00 - $20.00
Demand Coefficients:
  α (Occupancy): 0.4
  β (Queue): 0.15
  γ (Traffic): 0.1
  δ (Special Day): 0.25
  ε (Vehicle Type): 0.1
  λ (Price Sensitivity): 0.8
Formula: Demand = α·(Occ/Cap) + β·Queue - γ·Traffic + δ·Holiday + ε·Vehicle
Price = BasePrice × (1 + λ × NormalizedDemand)


In [36]:
# Step 13: Analysis Functions
def analyze_model2_behavior():
    """Analyze Model 2 pricing behavior"""
    print("\n=== MODEL 2 BEHAVIOR ANALYSIS ===")

    # Sample scenarios
    scenarios = [
        {"occupancy": 0.3, "queue": 0, "traffic": "Low", "vehicle": "Bike", "time": "morning"},
        {"occupancy": 0.8, "queue": 3, "traffic": "High", "vehicle": "Car", "time": "evening"},
        {"occupancy": 0.9, "queue": 5, "traffic": "High", "vehicle": "Truck", "time": "evening"},
        {"occupancy": 0.2, "queue": 0, "traffic": "Low", "vehicle": "Bike", "time": "night"}
    ]

    print("Sample pricing scenarios:")
    for i, scenario in enumerate(scenarios, 1):
        demand = calculate_demand_score(
            scenario["occupancy"], scenario["queue"],
            scenario["traffic"], scenario["vehicle"],
            "2024-01-15 08:00:00"  # Sample timestamp
        )
        normalized = normalize_demand(demand)
        price = BASE_PRICE * (1 + LAMBDA * normalized)
        price = max(BASE_PRICE * MIN_PRICE_MULTIPLIER,
                   min(price, BASE_PRICE * MAX_PRICE_MULTIPLIER))

        print(f"Scenario {i}: Occ={scenario['occupancy']:.1f}, "
              f"Queue={scenario['queue']}, Traffic={scenario['traffic']}, "
              f"Vehicle={scenario['vehicle']} → Price=${price:.2f}")

def get_model2_insights():
    """Generate insights for Model 2"""
    print("\n=== MODEL 2 INSIGHTS ===")
    print("After pipeline execution, analyze:")
    print("1. Price variations by time of day")
    print("2. Weekend vs weekday pricing patterns")
    print("3. Impact of vehicle type on pricing")
    print("4. Queue length correlation with prices")
    print("5. Traffic level effects on demand")
    print("6. Holiday pricing premiums")

# **Run Model 2 Pipeline**

In [38]:
# Step 14: Run Model 2 Pipeline
if __name__ == "__main__":
    print("Starting Model 2 - Advanced Demand-Based Pricing...")

    # Run analysis
    analyze_model2_behavior()
    get_model2_insights()
    pw.run()

    # Run the Pathway pipeline in a way that keeps the visual board active
    # Use `pw.run(loop=True)` for continuous processing (or similar mechanism
    # depending on your Pathway setup and desired behavior, e.g., integrating
    # with a web server). For this example, we'll use a simple run.
    # To keep a Panel app running, you typically don't use pw.run() directly
    # in a script like this. Instead, the Panel app is served, and Pathway
    # runs in the background to update the data source used by the Panel app.

    # In a Colab environment with Panel, serving the Panel object (viz or pn.Column(viz))
    # in the last cell is sufficient to start the server and the Pathway pipeline.
    # The previous cell already serves the Panel object.

    print("\n=== MODEL 2 EXECUTION COMPLETE ===")
    print("Generated files:")
    print("- model2_pricing_results.csv: Detailed pricing results")
    print("- model2_lot_summary.csv: Lot-wise summary statistics")
    print("- model2_time_analysis.csv: Time-based pricing analysis")
    print("- model2_day_analysis.csv: Day type analysis")
    print("- model2_vehicle_analysis.csv: Vehicle type analysis")

Output()

Starting Model 2 - Advanced Demand-Based Pricing...

=== MODEL 2 BEHAVIOR ANALYSIS ===
Sample pricing scenarios:
Scenario 1: Occ=0.3, Queue=0, Traffic=Low, Vehicle=Bike → Price=$11.14
Scenario 2: Occ=0.8, Queue=3, Traffic=High, Vehicle=Car → Price=$13.45
Scenario 3: Occ=0.9, Queue=5, Traffic=High, Vehicle=Truck → Price=$14.46
Scenario 4: Occ=0.2, Queue=0, Traffic=Low, Vehicle=Bike → Price=$10.77

=== MODEL 2 INSIGHTS ===
After pipeline execution, analyze:
1. Price variations by time of day
2. Weekend vs weekday pricing patterns
3. Impact of vehicle type on pricing
4. Queue length correlation with prices
5. Traffic level effects on demand
6. Holiday pricing premiums



=== MODEL 2 EXECUTION COMPLETE ===
Generated files:
- model2_pricing_results.csv: Detailed pricing results
- model2_lot_summary.csv: Lot-wise summary statistics
- model2_time_analysis.csv: Time-based pricing analysis
- model2_day_analysis.csv: Day type analysis
- model2_vehicle_analysis.csv: Vehicle type analysis
